In [63]:
# Biblioteki wejściowe
import pandas as pd
import numpy as np
import posixpath as os
import pathlib
import sys
import plotly.express as px
import plotly.io as pio
pd.options.plotting.backend = "plotly"
from IPython.display import display, HTML
import ipywidgets as widgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import codecs
from weasyprint import HTML, CSS
from pandas.tseries.offsets import MonthEnd

pd.options.display.max_rows=5000
pd.options.display.precision=2
pd.options.display.max_columns=500
pd.options.display.max_colwidth=500
pd.options.display.float_format='{0:,.2f}'.format

from datetime import datetime, timedelta

p=r'C:\Users\DW\KOPALNIA\FINANSE\SALE&PRODUCTION\DATA_SOURCE'

In [83]:
d1=pd.read_excel(os.join(p,r'\R_2104.xlsx'))


raport=pd.read_pickle(os.join(p,r'\Raport_Waga.pkl'))

d1['Unnamed: 0']=d1['Unnamed: 0'].str.lstrip()
d2=d1['Unnamed: 0'].str.split('\s{5,}',expand=True)
d3=pd.DataFrame()
for x in range(1,len(d2)):
    if x%9==0 and x+6<len(d2):
        Kontrahent=d2.iloc[x,0]
        Cena=d2.iloc[x+6,2]
        Ilość=d2.iloc[x+4,2]
        Data=d2.iloc[x+1,1]
        Przewoźnik=d2.iloc[x+2,0]
        Asortyment=d2.iloc[x+5,0]

        df=pd.DataFrame([[Kontrahent,Cena,Ilość,Data,Przewoźnik,Asortyment]])
        d3=pd.concat([d3,df],ignore_index=True)

d3.drop(d3[d3[0].isnull()==True].index,inplace=True)
d3[6]=d3[0].str.split('\s{2,}')
d3[7]=d3[5].str.split('Asortyment:')
d3[8]=d3[6].str[1]
d3[9]=d3[7].str[1]
d3=d3[[9,2,1,8,4,3]].copy()
d3.rename(columns={9:'Asortyment',2:'Ilość',1:'Cena',8:'Kontrahent',4:'Przewoźnik',3:'Data'},inplace=True)

d3['Ilość']=d3['Ilość'].str.rstrip()
d3['Ilość']=d3['Ilość'].str.replace(',','.')
d3['Ilość']=d3['Ilość'].astype(float)

d3['Cena']=d3['Cena'].str.rstrip()
d3['Cena']=d3['Cena'].str.replace(',','.')
d3['Cena']=d3['Cena'].astype(float)
d3['Data']=pd.to_datetime(d3['Data'],dayfirst=True)
d3['Data_M']=d3['Data'].dt.date
d3['M']=d3['Data'].dt.month

#raport=d3
raport=pd.concat([raport,d3],ignore_index=True)
raport.to_pickle(os.join(p,r'\Raport_Waga.pkl'))

In [84]:
raport=pd.read_pickle(os.join(p,r'\Raport_Waga.pkl'))
raport['Transport']=raport.apply(lambda x:'Kolejowy' if x['Kontrahent']=='BOCZNICA' else 'Drogowy',axis=1)
raport['D']=raport['Data'].dt.day

In [85]:
raport['CM']=pd.to_datetime("today").month
raport['CD']=pd.to_datetime("today").day

In [86]:
raport['Data_R']=raport.apply(lambda x:'Dzien (n-1)' if (x['CM']==x['M']) and (x['D']==x['CD']-1) else 'Od początku',axis=1)
raport['Data_R']=raport.apply(lambda x:'Dzien (n-2)' if (x['CM']==x['M']) and (x['D']==x['CD']-2) else x['Data_R'],axis=1)
raport['Data_R']=raport.apply(lambda x:'Dzien (n-3)' if (x['CM']==x['M']) and (x['D']==x['CD']-3) else x['Data_R'],axis=1)
raport['Data_R']=raport.apply(lambda x:'Dzien (n-4)' if (x['CM']==x['M']) and (x['D']==x['CD']-4) else x['Data_R'],axis=1)
raport['Data_R']=raport.apply(lambda x:'Dzien (n-5)' if (x['CM']==x['M']) and (x['D']==x['CD']-5) else x['Data_R'],axis=1)
raport['Data_R']=raport.apply(lambda x:'Dzien (n-6)' if (x['CM']==x['M']) and (x['D']==x['CD']-6) else x['Data_R'],axis=1)
raport['Data_R']=raport.apply(lambda x:'Dzien (n-7)' if (x['CM']==x['M']) and (x['D']==x['CD']-7) else x['Data_R'],axis=1)


In [82]:
raport.to_clipboard(decimal=',',excel=True,index=False)

In [87]:
raport_all=raport.groupby(['Data_M'],as_index=False)['Ilość'].sum()
raport_all['Ilość_YTD']=raport_all['Ilość'].cumsum()

In [92]:
raport_pkp=raport[raport['Kontrahent']=='BOCZNICA']
raport_pkp=raport_pkp.groupby(['Data_M'],as_index=False)['Ilość'].sum()
raport_pkp['Ilość_YTD']=raport_pkp['Ilość'].cumsum()

In [88]:
fig = raport_all.plot.line(x='Data_M',y='Ilość',title='Sprzedaż w tonach',width=800, height=500,markers=True)
fig.update_layout(plot_bgcolor='#F8F8F8',font_size=10,title_font_size=12)
fig.show()

In [93]:
fig = raport_pkp.plot.line(x='Data_M',y='Ilość',title='Sprzedaż w tonach',width=800, height=500,markers=True)
fig.update_layout(plot_bgcolor='#F8F8F8',font_size=10,title_font_size=12)
fig.show()

In [12]:
r1=raport
raport_asort=r1.groupby(['Asortyment'],as_index=False)['Ilość'].sum()
raport_asort=raport_asort.sort_values(by='Ilość')

In [13]:
fig = raport_asort.plot.bar(x='Asortyment',y='Ilość',title='Sprzedaż w tonach Asortymenty',width=800, height=500)
fig.update_layout(plot_bgcolor='#F8F8F8',font_size=10,title_font_size=12)
fig.show()